In [ ]:
# Establish accounts. Fill out variables here as needed. i.e. project/awardee/pmi_account/service_account
# Once you save this file, you don't have to retype these.

project = "all-of-us-rdr-stable" # type in environment if not stable
awardee = "KUMC" # fill in your awardee
pmi_account = "lav.patel@pmi-ops.org" # update your pmi-ops account.
service_account = "awardee-KUMC@all-of-us-ops-data-api-stable.iam.gserviceaccount.com" #update to your service account
!curl -6 https://ifconfig.co
!curl -4 https://ifconfig.co

In [ ]:
# Authentication. This cell creates a key for your service account, you don't need to do anything.
# If you are running this notebook on your local system (the legacy method) you can un-comment the following line to create
# a service account key that is called 'gcloud_key.json'. Otherwise you should have this key in the file structure already.
#results = !gcloud -q iam service-accounts keys create --account $pmi_account --project $project --iam-account $service_account gcloud_key.json
!gcloud -q auth activate-service-account --key-file=/Users/lpatel/.allofus/awardee-kumc_all-of-us-ops-data-api-stable_iam_gserviceaccount_com_key.json

In [ ]:
# Get Oauth Token, refresh as needed.
token = !gcloud -q auth print-access-token
token = token[0]
headers = {'content-type': 'application/json', 'Authorization': 'Bearer {0}'.format(token)}
print('Authentication Token Ready!') if token.startswith('ya') else 'Authentication Token Error!'

In [ ]:
# Make request to get API version. This is the current RDR version for reference
import requests

# disable ipv6 support
requests.packages.urllib3.util.connection.HAS_IPV6 = False

resp = requests.get('http://all-of-us-rdr-stable.appspot.com/rdr/v1/', headers=headers)
print(resp.json())

In [ ]:
# Request first 25 participants by default. You can change parameters of the url as needed.
# See https://github.com/all-of-us/raw-data-repository/blob/master/opsdataAPI.md for documentation of this api.
count = 25
url = 'https://{0}.appspot.com/rdr/v1/ParticipantSummary?_sort=lastModified&_count={1}&awardee={2}'.format(project, count, awardee)

resp = requests.get(url, headers=headers)
if not resp or resp.status_code != 200:
    print(resp.text)
    print('Error: api request failed.\n\n{0}.'.format(resp.text if resp else 'Unknown error.'))
else:
    ps_data = resp.json()
    print('Success: retrieved {0} records.'.format(len(ps_data['entry'])))

In [ ]:
# This cell creates a dataframe to show a graphical representation of what is returned.

from datetime import datetime
import pandas
import matplotlib.pyplot as plt
%matplotlib inline 

good_cols = ['ageRange', 'dateOfBirth', 'participantId', 'race', 'sex']

data = []

# loop over participant summary records, insert participant data in same order as good_cols.
for entry in ps_data['entry']:
    item = []
    for col in good_cols:
        for key, val in entry['resource'].items():
            if col == key:
                if key == 'dateOfBirth':                    
                    item.append(datetime.strptime(val, '%Y-%m-%d'))
                else:
                    item.append(val)    
    data.append(item)
    
frame = pandas.DataFrame(data, columns=good_cols)

plt.hist(frame["dateOfBirth"].dt.weekday, bins=6)


In [ ]:
# Deletes the google cloud key created in first step so that you don't hit the Google enforced limit of 10 keys.
# We don't typically need this because we no longer create new keys each time it's run.
#import os
#os.remove('gcloud_key.json')
#!gcloud -q iam service-accounts keys delete $key_id --account $pmi_account --iam-account $service_account

### NOTES:
* Cells will be added/modified by the dev team while working with data stewards to determine specific needs.
* You are welcome to add cells to view info in different ways if you're comfortable with Python
    * if you ever want to return this notebook to it's original state type `git checkout -- ops_data_api.ipynb` from the ops_data_api directory.